In [1]:
import pandas as pd
import numpy as np
from google.colab import files
uploaded = files.upload()      #MovieInformation.csv, RatingData.csv, UserInformation.csv

Saving MovieInformation.csv to MovieInformation.csv


In [3]:
Movies= pd.read_csv("MovieInformation.csv") 
print("Total movies are:", Movies['MovieID'].count())
Movies.head()

Total movies are: 1682


,MovieID,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,FilmNoir,Horror,Musical,Mystery,Romance,SciFi,Thriller,War,Western
0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [ ]:
Ratings= pd.read_csv("RatingData.csv") 
Ratings = Ratings.drop('Timestamp',axis=1)
print("Total Ratings are:", Ratings['MovieID'].count())
#Ratings.head()
#Ratings.loc[Ratings['UserID'] == 196]

Total Ratings are: 100000


In [ ]:
AllGenres= ['unknown', 'Action', 'Adventure', 'Animation', 'Children',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'FilmNoir',
       'Horror', 'Musical', 'Mystery', 'Romance', 'SciFi', 'Thriller',
       'War', 'Western']
#AllGenres

In [ ]:
MergedRatingsMovieData = pd.merge(left=Ratings, right=Movies, how='left', left_on='MovieID', right_on='MovieID')
MergedRatingsMovieData.replace(0, np.nan, inplace=True)   #replacing 0s with Nan so that 0 values are not counted when computing the average rating
#MergedRatingsMovieData.loc[MergedRatingsMovieData['MovieID'] == 1]
#Ratings.loc[Ratings['MovieID'] == 663]
MergedRatingsMovieData
MergedRatingsMovieData.loc[MergedRatingsMovieData['UserID'] == 1]

,UserID,MovieID,Ratings,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,FilmNoir,Horror,Musical,Mystery,Romance,SciFi,Thriller,War,Western
202,1,61,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
305,1,189,3,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
333,1,33,4,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN
334,1,160,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
478,1,20,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92049,1,28,4,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
92487,1,172,5,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,NaN
94019,1,122,3,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96699,1,152,5,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN


In [ ]:
AugmentedRatings = MergedRatingsMovieData.copy()
AllGenresRatings = []
for genre in AllGenres:
   AllGenresRatings.append(genre+'Rating')
   AugmentedRatings[genre+'Rating'] = AugmentedRatings[genre] * AugmentedRatings['Ratings']  #each genre gets the same rating for that movie
#print(AllGenresRatings)
AugmentedRatings.head()

,UserID,MovieID,Ratings,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,FilmNoir,Horror,Musical,Mystery,Romance,SciFi,Thriller,War,Western,unknownRating,ActionRating,AdventureRating,AnimationRating,ChildrenRating,ComedyRating,CrimeRating,DocumentaryRating,DramaRating,FantasyRating,FilmNoirRating,HorrorRating,MusicalRating,MysteryRating,RomanceRating,SciFiRating,ThrillerRating,WarRating,WesternRating
0,196,242,3,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,186,302,3,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,3.0,NaN,NaN,3.0,NaN,NaN,3.0,NaN,NaN
2,22,377,1,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,244,51,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,2.0,2.0
4,166,346,1,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#grouping the data genre columns now have genreCount, and genreRatigs have genreSumOfRatings for that genre
PerUserGroupedMoviesData = AugmentedRatings.groupby(by=["UserID"],as_index=False).sum()   #The argument as_index is to configure whether the index is group labels or not. If it is set to False, the group labels are represented as columns instead of index.
#result.reset_index()     #did not work
PerUserGroupedMoviesData['NumMoviesWatched'] = PerUserGroupedMoviesData[AllGenres].sum(axis=1)
PerUserGroupedMoviesData

,UserID,MovieID,Ratings,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,FilmNoir,Horror,Musical,Mystery,Romance,SciFi,Thriller,War,Western,unknownRating,ActionRating,AdventureRating,AnimationRating,ChildrenRating,ComedyRating,CrimeRating,DocumentaryRating,DramaRating,FantasyRating,FilmNoirRating,HorrorRating,MusicalRating,MysteryRating,RomanceRating,SciFiRating,ThrillerRating,WarRating,WesternRating,NumMoviesWatched
0,1,37128,982,1.0,75.0,42.0,12.0,25.0,91.0,25.0,5.0,107.0,2.0,1.0,13.0,13.0,5.0,44.0,43.0,52.0,25.0,6.0,4.0,250.0,123.0,40.0,55.0,316.0,86.0,24.0,420.0,7.0,5.0,45.0,38.0,18.0,173.0,172.0,188.0,92.0,22.0,587.0
1,2,15469,230,0.0,10.0,3.0,1.0,4.0,16.0,9.0,0.0,35.0,1.0,2.0,2.0,1.0,4.0,16.0,4.0,12.0,3.0,0.0,0.0,38.0,13.0,4.0,12.0,61.0,34.0,0.0,134.0,3.0,9.0,6.0,3.0,14.0,66.0,15.0,43.0,11.0,0.0,123.0
2,3,17216,151,0.0,14.0,4.0,0.0,0.0,12.0,10.0,1.0,22.0,0.0,2.0,5.0,2.0,11.0,5.0,8.0,21.0,5.0,0.0,0.0,39.0,14.0,0.0,0.0,31.0,30.0,5.0,64.0,0.0,5.0,12.0,4.0,35.0,17.0,22.0,53.0,14.0,0.0,122.0
3,4,6985,104,0.0,8.0,4.0,0.0,0.0,4.0,4.0,1.0,6.0,0.0,0.0,1.0,1.0,5.0,3.0,6.0,11.0,2.0,0.0,0.0,31.0,14.0,0.0,0.0,20.0,19.0,5.0,27.0,0.0,0.0,4.0,5.0,20.0,13.0,23.0,43.0,9.0,0.0,56.0
4,5,50976,503,1.0,56.0,33.0,14.0,29.0,82.0,9.0,0.0,27.0,2.0,1.0,28.0,12.0,3.0,19.0,33.0,19.0,14.0,2.0,4.0,176.0,107.0,53.0,71.0,246.0,35.0,0.0,72.0,5.0,5.0,71.0,40.0,9.0,44.0,116.0,56.0,45.0,5.0,384.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,939,25190,209,0.0,18.0,10.0,1.0,1.0,15.0,3.0,0.0,18.0,1.0,0.0,1.0,1.0,1.0,10.0,8.0,12.0,3.0,0.0,0.0,73.0,35.0,4.0,4.0,68.0,13.0,0.0,82.0,4.0,0.0,3.0,4.0,4.0,48.0,33.0,49.0,15.0,0.0,103.0
939,940,37113,370,0.0,23.0,12.0,2.0,5.0,42.0,7.0,0.0,48.0,0.0,3.0,4.0,7.0,3.0,24.0,19.0,20.0,15.0,0.0,0.0,79.0,36.0,9.0,21.0,148.0,30.0,0.0,173.0,0.0,12.0,12.0,22.0,13.0,85.0,58.0,67.0,49.0,0.0,234.0
940,941,7872,89,0.0,10.0,7.0,3.0,2.0,7.0,1.0,0.0,5.0,0.0,0.0,0.0,1.0,1.0,1.0,8.0,7.0,1.0,0.0,0.0,38.0,27.0,14.0,9.0,29.0,3.0,0.0,21.0,0.0,0.0,0.0,4.0,5.0,5.0,31.0,28.0,5.0,0.0,54.0
941,942,33900,337,0.0,18.0,11.0,4.0,13.0,22.0,0.0,0.0,31.0,2.0,1.0,3.0,5.0,9.0,17.0,6.0,20.0,10.0,3.0,0.0,74.0,52.0,19.0,56.0,90.0,0.0,0.0,137.0,8.0,5.0,11.0,22.0,36.0,75.0,25.0,80.0,47.0,14.0,175.0


In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

NormalizedPerUserMovieData = PerUserGroupedMoviesData.copy()
for genre in AllGenres:   #calculating average rating pergenre
   NormalizedPerUserMovieData[genre+'Rating'] = NormalizedPerUserMovieData[genre+'Rating'] / (NormalizedPerUserMovieData[genre]*5.0) 
   #NormalizedPerUserMovieData[genre+'Rating'] = NormalizedPerUserMovieData[genre+'Rating'] / (NormalizedPerUserMovieData['NumMoviesWatched'] *5.0) 
#normalizing number of movies watched
#NormalizedPerUserMovieData[AllGenresRatings].div(NormalizedPerUserMovieData['NumMoviesWatched'],axis=0) //not working, don't know why
for genre in AllGenres:   
   NormalizedPerUserMovieData[genre] = NormalizedPerUserMovieData[genre] / NormalizedPerUserMovieData['NumMoviesWatched'] 

#create genrePreference (binary column) based on genreRating
for genre in AllGenres:
  NormalizedPerUserMovieData[genre+'Preference'] = 0
for genre in AllGenres:
  NormalizedPerUserMovieData[genre+'Preference'].loc[NormalizedPerUserMovieData[genre+'Rating'] >= 0.6] = 1   #ie average rating >= 3 for that genre

NormalizedPerUserMovieData = NormalizedPerUserMovieData.drop(['MovieID','Ratings','unknown','unknownRating'],axis=1)
NormalizedPerUserMovieData.replace(np.nan, 0, inplace=True)
NormalizedPerUserMovieData

,UserID,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,FilmNoir,Horror,Musical,Mystery,Romance,SciFi,Thriller,War,Western,ActionRating,AdventureRating,AnimationRating,ChildrenRating,ComedyRating,CrimeRating,DocumentaryRating,DramaRating,FantasyRating,FilmNoirRating,HorrorRating,MusicalRating,MysteryRating,RomanceRating,SciFiRating,ThrillerRating,WarRating,WesternRating,NumMoviesWatched,unknownPreference,ActionPreference,AdventurePreference,AnimationPreference,ChildrenPreference,ComedyPreference,CrimePreference,DocumentaryPreference,DramaPreference,FantasyPreference,FilmNoirPreference,HorrorPreference,MusicalPreference,MysteryPreference,RomancePreference,SciFiPreference,ThrillerPreference,WarPreference,WesternPreference
0,1,0.127768,0.071550,0.020443,0.042589,0.155026,0.042589,0.008518,0.182283,0.003407,0.001704,0.022147,0.022147,0.008518,0.074957,0.073254,0.088586,0.042589,0.010221,0.666667,0.585714,0.666667,0.440000,0.694505,0.688000,0.96,0.785047,0.7,1.0,0.692308,0.584615,0.720000,0.786364,0.800000,0.723077,0.736000,0.733333,587.0,1,1,0,1,0,1,1,1,1,1,1,1,0,1,1,1,1,1,1
1,2,0.081301,0.024390,0.008130,0.032520,0.130081,0.073171,0.000000,0.284553,0.008130,0.016260,0.016260,0.008130,0.032520,0.130081,0.032520,0.097561,0.024390,0.000000,0.760000,0.866667,0.800000,0.600000,0.762500,0.755556,0.00,0.765714,0.6,0.9,0.600000,0.600000,0.700000,0.825000,0.750000,0.716667,0.733333,0.000000,123.0,0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0
2,3,0.114754,0.032787,0.000000,0.000000,0.098361,0.081967,0.008197,0.180328,0.000000,0.016393,0.040984,0.016393,0.090164,0.040984,0.065574,0.172131,0.040984,0.000000,0.557143,0.700000,0.000000,0.000000,0.516667,0.600000,1.00,0.581818,0.0,0.5,0.480000,0.400000,0.636364,0.680000,0.550000,0.504762,0.560000,0.000000,122.0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0,0
3,4,0.142857,0.071429,0.000000,0.000000,0.071429,0.071429,0.017857,0.107143,0.000000,0.000000,0.017857,0.017857,0.089286,0.053571,0.107143,0.196429,0.035714,0.000000,0.775000,0.700000,0.000000,0.000000,1.000000,0.950000,1.00,0.900000,0.0,0.0,0.800000,1.000000,0.800000,0.866667,0.766667,0.781818,0.900000,0.000000,56.0,0,1,1,0,0,1,1,1,1,0,0,1,1,1,1,1,1,1,0
4,5,0.145833,0.085938,0.036458,0.075521,0.213542,0.023438,0.000000,0.070312,0.005208,0.002604,0.072917,0.031250,0.007812,0.049479,0.085938,0.049479,0.036458,0.005208,0.628571,0.648485,0.757143,0.489655,0.600000,0.777778,0.00,0.533333,0.5,1.0,0.507143,0.666667,0.600000,0.463158,0.703030,0.589474,0.642857,0.500000,384.0,1,1,1,1,0,1,1,0,0,0,1,0,1,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,939,0.174757,0.097087,0.009709,0.009709,0.145631,0.029126,0.000000,0.174757,0.009709,0.000000,0.009709,0.009709,0.009709,0.097087,0.077670,0.116505,0.029126,0.000000,0.811111,0.700000,0.800000,0.800000,0.906667,0.866667,0.00,0.911111,0.8,0.0,0.600000,0.800000,0.800000,0.960000,0.825000,0.816667,1.000000,0.000000,103.0,0,1,1,1,1,1,1,0,1,1,0,1,1,1,1,1,1,1,0
939,940,0.098291,0.051282,0.008547,0.021368,0.179487,0.029915,0.000000,0.205128,0.000000,0.012821,0.017094,0.029915,0.012821,0.102564,0.081197,0.085470,0.064103,0.000000,0.686957,0.600000,0.900000,0.840000,0.704762,0.857143,0.00,0.720833,0.0,0.8,0.600000,0.628571,0.866667,0.708333,0.610526,0.670000,0.653333,0.000000,234.0,0,1,1,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,0
940,941,0.185185,0.129630,0.055556,0.037037,0.129630,0.018519,0.000000,0.092593,0.000000,0.000000,0.000000,0.018519,0.018519,0.018519,0.148148,0.129630,0.018519,0.000000,0.760000,0.771429,0.933333,0.900000,0.828571,0.600000,0.00,0.840000,0.0,0.0,0.000000,0.800000,1.000000,1.000000,0.775000,0.800000,1.000000,0.000000,54.0,0,1,1,1,1,1,1,0,1,0,0,0,1,1,1,1,1,1,0
941,942,0.102857,0.062857,0.022857,0.074286,0.125714,0.000000,0.000000,0.177143,0.011429,0.005714,0.017143,0.028571,0.051429,0.097143,0.0

In [ ]:
UserInformation= pd.read_csv("UserInformation.csv") 
print("Total Users are:", UserInformation['UserID'].count())
mean_age = UserInformation.Age.dropna().mean()
max_age = UserInformation.Age.dropna().max()
min_age = UserInformation.Age.dropna().min()
print(mean_age, max_age, min_age)
UserInformation.head() 

Total Users are: 943
34.05196182396607 73 7


,UserID,Age,Gender,Occupation,Zipcode
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [ ]:
#UserInformation.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)    #There was an unamed column in selected users, needed to delete that column
#UserInformation.drop(["a"], axis=1, inplace=True)
#UserInformation.head()

In [ ]:
#UserInformation['Age'] = UserInformation['Age'].apply(lambda x: (x - mean_age ) / (max_age - min_age ))
UserInformation['NormalizedAge'] = UserInformation['Age'].apply(lambda x: (x/max_age ))  #normailize age

UserInformation['Gender'].replace('F',0,inplace=True)
UserInformation['Gender'].replace('M',1,inplace=True)  #convert categorical Gender to numeric

UserInformation['Male'] = 0
UserInformation['Female'] = 0

UserInformation['Male'].loc[UserInformation['Gender'] == 1] = 1
UserInformation['Female'].loc[UserInformation['Gender'] == 0] = 1

bins= [10,20,30,50,60,100]
AgeLabels = ['Teenager','Youth','MiddleAge','Senior','SuperSenior']
UserInformation['AgeGroup'] = pd.cut(UserInformation['Age'], bins=bins, labels=AgeLabels, right=False)

UserInformation.head()

,UserID,Age,Gender,Occupation,Zipcode,NormalizedAge,AgeGroup,Male,Female
0,1,24,1,technician,85711,0.328767,Youth,1,0
1,2,53,0,other,94043,0.726027,Senior,0,1
2,3,23,1,writer,32067,0.315068,Youth,1,0
3,4,24,1,technician,43537,0.328767,Youth,1,0
4,5,33,0,other,15213,0.452055,MiddleAge,0,1


In [ ]:
!pip install uszipcode

     |████████████████████████████████| 454 kB 30.1 MB/s 
     |████████████████████████████████| 78 kB 7.8 MB/s 
     |████████████████████████████████| 45 kB 3.0 MB/s 
     |████████████████████████████████| 41 kB 535 kB/s 


In [ ]:
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=True) # set simple_zipcode=False to use rich info database
search.by_zipcode('85711').to_dict()['state']
UserInformation['State'] = UserInformation.apply(lambda row: search.by_zipcode(row.Zipcode).to_dict()['state'], axis = 1)
#print(UserInformation['State'].nunique())
#print(UserInformation['State'].value_counts())
#UserInformation
#print(UserInformation['Occupation'].value_counts())

Start downloading data for simple zipcode database, total size 9MB ...
  1 MB finished ...
  2 MB finished ...
  3 MB finished ...
  4 MB finished ...
  5 MB finished ...
  6 MB finished ...
  7 MB finished ...
  8 MB finished ...
  9 MB finished ...
  10 MB finished ...
  Complete!


In [ ]:
AugmentedUserData = UserInformation.copy()           #using dummies method to convert categorical ocuupations to equal number of binary columns
#Occupations= pd.read_csv("OccupationTypes.csv")
#AllOccupations = Occupations['Occupation Types'].to_numpy()
AllOccupations = ['administrator', 'artist', 'doctor', 'educator', 'engineer',
       'entertainment', 'executive', 'healthcare', 'homemaker', 'lawyer',
       'librarian', 'marketing', 'none', 'other', 'programmer', 'retired',
       'salesman', 'scientist', 'student', 'technician', 'writer']
#AllOccupations

for label in AgeLabels:
  AugmentedUserData[label] = 0
for label in AgeLabels:
  AugmentedUserData[label].loc[AugmentedUserData['AgeGroup'] == label] = 1

for occupation in AllOccupations:
  AugmentedUserData[occupation] = 0

for occupation in AllOccupations:
  AugmentedUserData[occupation].loc[AugmentedUserData['Occupation'] == occupation] = 1

AugmentedUserData.loc[AugmentedUserData['UserID'] == 1]

,UserID,Age,Gender,Occupation,Zipcode,NormalizedAge,AgeGroup,Male,Female,State,Teenager,Youth,MiddleAge,Senior,SuperSenior,administrator,artist,doctor,educator,engineer,entertainment,executive,healthcare,homemaker,lawyer,librarian,marketing,none,other,programmer,retired,salesman,scientist,student,technician,writer
0,1,24,1,technician,85711,0.328767,Youth,1,0,AZ,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [ ]:
AllStates = pd.unique(UserInformation['State'])
#print(AllStates)       #using dummies method to convert categorical states to equal number of binary columns
for state in AllStates:
  AugmentedUserData[state] = 0

for state in AllStates:
  AugmentedUserData[state].loc[AugmentedUserData['State'] == state] = 1
#AugmentedUserData
AugmentedUserData.loc[AugmentedUserData['UserID'] == 1]

,UserID,Age,Gender,Occupation,Zipcode,NormalizedAge,AgeGroup,Male,Female,State,Teenager,Youth,MiddleAge,Senior,SuperSenior,administrator,artist,doctor,educator,engineer,entertainment,executive,healthcare,homemaker,lawyer,librarian,marketing,none,other,programmer,retired,salesman,scientist,student,technician,writer,AZ,CA,FL,OH,...,OR,NY,TN,KY,MI,MD,TX,NC,WI,CO,IN,IA,NJ,NaN,UT,IL,VA,OK,NH,DC,NV,NE,LA,MO,KS,AK,AL,ID,ND,MS,AR,MT,NM,RI,HI,WV,ME,DE,WY,SD
0,1,24,1,technician,85711,0.328767,Youth,1,0,AZ,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
FinalUserData = AugmentedUserData.copy()
#FinalUserData = FinalUserData.drop(['Occupation','Zipcode','State','Age','AgeGroup'],axis=1)
#FinalUserData.head()
FinalUserData.loc[FinalUserData['UserID'] == 1]

,UserID,Age,Gender,Occupation,Zipcode,NormalizedAge,AgeGroup,Male,Female,State,Teenager,Youth,MiddleAge,Senior,SuperSenior,administrator,artist,doctor,educator,engineer,entertainment,executive,healthcare,homemaker,lawyer,librarian,marketing,none,other,programmer,retired,salesman,scientist,student,technician,writer,AZ,CA,FL,OH,...,OR,NY,TN,KY,MI,MD,TX,NC,WI,CO,IN,IA,NJ,NaN,UT,IL,VA,OK,NH,DC,NV,NE,LA,MO,KS,AK,AL,ID,ND,MS,AR,MT,NM,RI,HI,WV,ME,DE,WY,SD
0,1,24,1,technician,85711,0.328767,Youth,1,0,AZ,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
NodeAttributes = pd.merge(left=FinalUserData, right=NormalizedPerUserMovieData, how='left', left_on='UserID', right_on='UserID') #NormalizedAverageRatings
#NodeAttributes.loc[NodeAttributes['UserID'] == 1]
print(NodeAttributes['UserID'].nunique())
NodeAttributes    #943 rows × 144 columns

943


,UserID,Age,Gender,Occupation,Zipcode,NormalizedAge,AgeGroup,Male,Female,State,Teenager,Youth,MiddleAge,Senior,SuperSenior,administrator,artist,doctor,educator,engineer,entertainment,executive,healthcare,homemaker,lawyer,librarian,marketing,none,other,programmer,retired,salesman,scientist,student,technician,writer,AZ,CA,FL,OH,...,War,Western,ActionRating,AdventureRating,AnimationRating,ChildrenRating,ComedyRating,CrimeRating,DocumentaryRating,DramaRating,FantasyRating,FilmNoirRating,HorrorRating,MusicalRating,MysteryRating,RomanceRating,SciFiRating,ThrillerRating,WarRating,WesternRating,NumMoviesWatched,unknownPreference,ActionPreference,AdventurePreference,AnimationPreference,ChildrenPreference,ComedyPreference,CrimePreference,DocumentaryPreference,DramaPreference,FantasyPreference,FilmNoirPreference,HorrorPreference,MusicalPreference,MysteryPreference,RomancePreference,SciFiPreference,ThrillerPreference,WarPreference,WesternPreference
0,1,24,1,technician,85711,0.328767,Youth,1,0,AZ,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,...,0.042589,0.010221,0.666667,0.585714,0.666667,0.440000,0.694505,0.688000,0.96,0.785047,0.7,1.0,0.692308,0.584615,0.720000,0.786364,0.800000,0.723077,0.736000,0.733333,587.0,1,1,0,1,0,1,1,1,1,1,1,1,0,1,1,1,1,1,1
1,2,53,0,other,94043,0.726027,Senior,0,1,CA,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,...,0.024390,0.000000,0.760000,0.866667,0.800000,0.600000,0.762500,0.755556,0.00,0.765714,0.6,0.9,0.600000,0.600000,0.700000,0.825000,0.750000,0.716667,0.733333,0.000000,123.0,0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0
2,3,23,1,writer,32067,0.315068,Youth,1,0,FL,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,...,0.040984,0.000000,0.557143,0.700000,0.000000,0.000000,0.516667,0.600000,1.00,0.581818,0.0,0.5,0.480000,0.400000,0.636364,0.680000,0.550000,0.504762,0.560000,0.000000,122.0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0,0
3,4,24,1,technician,43537,0.328767,Youth,1,0,OH,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,...,0.035714,0.000000,0.775000,0.700000,0.000000,0.000000,1.000000,0.950000,1.00,0.900000,0.0,0.0,0.800000,1.000000,0.800000,0.866667,0.766667,0.781818,0.900000,0.000000,56.0,0,1,1,0,0,1,1,1,1,0,0,1,1,1,1,1,1,1,0
4,5,33,0,other,15213,0.452055,MiddleAge,0,1,PA,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0.036458,0.005208,0.628571,0.648485,0.757143,0.489655,0.600000,0.777778,0.00,0.533333,0.5,1.0,0.507143,0.666667,0.600000,0.463158,0.703030,0.589474,0.642857,0.500000,384.0,1,1,1,1,0,1,1,0,0,0,1,0,1,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,939,26,0,student,33319,0.356164,Youth,0,1,FL,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,...,0.029126,0.000000,0.811111,0.700000,0.800000,0.800000,0.906667,0.866667,0.00,0.911111,0.8,0.0,0.600000,0.800000,0.800000,0.960000,0.825000,0.816667,1.000000,0.000000,103.0,0,1,1,1,1,1,1,0,1,1,0,1,1,1,1,1,1,1,0
939,940,32,1,administrator,2215,0.438356,MiddleAge,1,0,MA,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.064103,0.000000,0.686957,0.600000,0.900000,0.840000,0.704762,0.857143,0.00,0.720833,0.0,0.8,0.600000,0.628571,0.866667,0.708333,0.610526,0.670000,0.653333,0.000000,234.0,0,1,1,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,0
940,941,20,1,student,97229,0.273973,Youth,1,0,OR,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0.018519,0.000000,0.760000,0.771429,0.933333,0.900000,0.828571,0.600000,0.00,0.840000,0.0,0.0,0.000000,0.800000,1.000000,1.000000,0.775000,0.800000,1.000000,0.000000,54.0,0,1,1,1,1,1,1,0,1,0,0,0,1,1,1,1,1,1,0
941,942,48,0,librarian,78209,0.657534,MiddleAge,0,1,TX,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.057143,0.017143,0.82222

In [ ]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [ ]:
NodeAttributes.to_csv("/drive/My Drive/Colab Notebooks/MovieLens-recommender/Data/AllNodeAttributes.csv")